Se transforman los datos desde Stata a CSV

In [1]:
import pandas as pd

# Leer sin convertir categorías
df = pd.read_stata("casen_en_Pandemia_2020_STATA.dta", convert_categoricals=False)

reader = pd.io.stata.StataReader("casen_en_Pandemia_2020_STATA.dta")
value_labels = reader.value_labels()
reader.close()

# Guardar
df.to_csv("CASEN_PANDEMIA_2020.csv", index=False, encoding='utf-8-sig')
print(f"{len(df)} filas guardadas")

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\835817429.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata("casen_en_Pandemia_2020_STATA.dta", convert_categoricals=False)
C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\835817429.py:8: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future version without notice. Using StataReader as a context manager is the only supported method.
  reader.close()


185437 filas guardadas


Inspeccionar los datos del cansen para evaluarlos

In [2]:
df.head()



,folio,o,id_persona,id_vivienda,region,provincia,comuna,zona,area,segmento,...,v20,v20_esp,v20_red,v22,v23,v23_sistema,v23_cajon,v27,v28,v29
0,1.101100e+11,1,5,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,2,NaN,NaN,1,NaN,NaN
1,1.101100e+11,2,6,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,2,NaN,NaN,1,NaN,NaN
2,1.101100e+11,1,32,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN
3,1.101100e+11,2,31,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN
4,1.101100e+11,3,30,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN


Ver datos de pobreza

In [3]:
print(df['pobreza'].value_counts())

pobreza
3.0    164042
2.0     12862
1.0      8435
Name: count, dtype: int64


In [4]:
print('pobreza' in df.columns)

True


In [5]:
print(df['pobreza_sinte'].value_counts())

pobreza_sinte
3    158565
2     15292
1     11580
Name: count, dtype: int64


In [6]:
print('nse' in df.columns)

False


In [8]:
import numpy as np
mapeo_texto = {
    1: 'Bajo',
    2: 'Medio', 
    3: 'Alto',
    4: 'Bajo-medio',
    5: 'Bajo-alto',
    6: 'Bajo-medio-alto',
    7: 'Medio-alto'
}

conditions = [
    df['pobreza_sinte'] == 1,                                      # 1 - Bajo (Indigentes)
    df['pobreza_sinte'] == 2,                                      # 4 - Bajo-medio (Pobres)
    (df['pobreza_sinte'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.90)),  # 3 - Alto
    (df['pobreza_sinte'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.75)),  # 7 - Medio-alto
    (df['pobreza_sinte'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.60)),  # 6 - Bajo-medio-alto
    (df['pobreza_sinte'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.40)),  # 2 - Medio
    (df['pobreza_sinte'] == 3) & (df['ypchautcor'] >= df['ypchautcor'].quantile(0.25)),  # 5 - Bajo-alto
]

choices = [1, 4, 3, 7, 6, 2, 5]

df['nse_calculado'] = np.select(conditions, choices, default=4)  # Por defecto Bajo-medio
df['Nivel socioeconomico'] = df['nse_calculado'].map(mapeo_texto)

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\2259049735.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['nse_calculado'] = np.select(conditions, choices, default=4)  # Por defecto Bajo-medio
C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\2259049735.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Nivel socioeconomico'] = df['nse_calculado'].map(mapeo_texto)


In [9]:
print(df['Nivel socioeconomico'].value_counts())

Nivel socioeconomico
Bajo-medio         38000
Medio              36864
Bajo-medio-alto    28071
Medio-alto         27815
Bajo-alto          24562
Alto               18545
Bajo               11580
Name: count, dtype: int64


In [10]:
mapeo_sistema_pro = {
    -88: 'no sabe',
    1: 'Fonasa', 
    2: 'Isapre',
    3: 'FF.AA. y del Orden',
    4: 'Ninguno (particular)',
    5: 'Otro sistema'
}

# Aplicar el mapeo
df['Sistema Provisional'] = df['s13'].map(mapeo_sistema_pro)

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\4171661879.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Sistema Provisional'] = df['s13'].map(mapeo_sistema_pro)


In [12]:
print('s13_fonasa' in df.columns)

False


In [13]:
mapeo_pobreza = {
    1: 'Pobreza Extrema',
    2: 'Pobreza No Extrema', 
    3: 'No Pobreza'
}
df['Pobreza'] = df['pobreza'].map(mapeo_pobreza)

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\3783429643.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pobreza'] = df['pobreza'].map(mapeo_pobreza)


In [14]:
mapeo_sexo = {
    1: 'Hombre',
    2: 'Mujer'
}

# Aplicar el mapeo
df['Sexo'] = df['sexo'].map(mapeo_sexo)

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\1727529262.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Sexo'] = df['sexo'].map(mapeo_sexo)


In [15]:
df['Año'] = df['fecha_entrev'].dt.year
df['Ingreso por Hogar'] = df['ytotcorh']

C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\4232368506.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Año'] = df['fecha_entrev'].dt.year
C:\Users\Liz\AppData\Local\Temp\ipykernel_3712\4232368506.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Ingreso por Hogar'] = df['ytotcorh']


Ya investigado la documentacion entregada por la web de la base de datos y el dataframe, tenemos el nombre de las variables para crear un dataframe de pobreza con CASEN EN PANDEMIA 2020

In [22]:
# Variables correctas disponibles
variables_necesarias = ['folio', 'id_persona','comuna','Pobreza', 'Nivel socioeconomico','pobreza_sinte','region','Año','Sistema Provisional',
                          'edad', 'Sexo', 'Ingreso por Hogar']

df_pobreza = df[variables_necesarias].copy()

# Eliminar filas donde pobreza sea nula
df_pobreza = df_pobreza.dropna(subset=['Pobreza'])

# Ver distribución de pobreza
print(" DISTRIBUCIÓN DE POBREZA POR INGRESOS ")
print(df_pobreza['Pobreza'].value_counts().sort_index())

print("\n DISTRIBUCIÓN DE Pobreza por Sistema de Imputación de Ingresos No Trabajo y Empleo")
print(df_pobreza['pobreza_sinte'].value_counts().sort_index())

print(f"\nTotal personas con datos: {len(df_pobreza)}")

 DISTRIBUCIÓN DE POBREZA POR INGRESOS 
Pobreza
No Pobreza            164042
Pobreza Extrema         8435
Pobreza No Extrema     12862
Name: count, dtype: int64

 DISTRIBUCIÓN DE Pobreza por Sistema de Imputación de Ingresos No Trabajo y Empleo
pobreza_sinte
1     11580
2     15292
3    158467
Name: count, dtype: int64

Total personas con datos: 185339


In [23]:
df_pobreza.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185339 entries, 0 to 185436
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   folio                 185339 non-null  float64
 1   id_persona            185339 non-null  int16  
 2   comuna                185339 non-null  int16  
 3   Pobreza               185339 non-null  object 
 4   Nivel socioeconomico  185339 non-null  object 
 5   pobreza_sinte         185339 non-null  int8   
 6   region                185339 non-null  int8   
 7   Año                   185339 non-null  int32  
 8   Sistema Provisional   182937 non-null  object 
 9   edad                  185339 non-null  int16  
 10  Sexo                  185339 non-null  object 
 11  Ingreso por Hogar     185339 non-null  float64
dtypes: float64(2), int16(3), int32(1), int8(2), object(4)
memory usage: 12.0+ MB


In [24]:
df_pobreza.head()

,folio,id_persona,comuna,Pobreza,Nivel socioeconomico,pobreza_sinte,region,Año,Sistema Provisional,edad,Sexo,Ingreso por Hogar
0,1.101100e+11,5,1101,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,34,Mujer,390833.0
1,1.101100e+11,6,1101,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,4,Mujer,390833.0
2,1.101100e+11,32,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Fonasa,45,Hombre,947583.0
3,1.101100e+11,31,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),5,Mujer,947583.0
4,1.101100e+11,30,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),19,Mujer,947583.0


Ahora vamos a codificar las comunas con codigos_geograficos_casen2022

In [25]:
df_codigo_geo = pd.read_csv('codigos_geograficos_casen2022.csv')

In [26]:
df_final = df_pobreza.merge(
    df_codigo_geo,
    left_on='comuna', 
    right_on='codigo_comuna',
    how='left'
)

In [27]:
df_final.head()

,folio,id_persona,comuna,Pobreza,Nivel socioeconomico,pobreza_sinte,region,Año,Sistema Provisional,edad,Sexo,Ingreso por Hogar,codigo_comuna,nombre_comuna,codigo_provincia,nombre_provincia
0,1.101100e+11,5,1101,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,34,Mujer,390833.0,1101,Iquique,11,Iquique
1,1.101100e+11,6,1101,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,4,Mujer,390833.0,1101,Iquique,11,Iquique
2,1.101100e+11,32,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Fonasa,45,Hombre,947583.0,1101,Iquique,11,Iquique
3,1.101100e+11,31,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),5,Mujer,947583.0,1101,Iquique,11,Iquique
4,1.101100e+11,30,1101,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),19,Mujer,947583.0,1101,Iquique,11,Iquique


In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185339 entries, 0 to 185338
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   folio                 185339 non-null  float64
 1   id_persona            185339 non-null  int16  
 2   comuna                185339 non-null  int16  
 3   Pobreza               185339 non-null  object 
 4   Nivel socioeconomico  185339 non-null  object 
 5   pobreza_sinte         185339 non-null  int8   
 6   region                185339 non-null  int8   
 7   Año                   185339 non-null  int32  
 8   Sistema Provisional   182937 non-null  object 
 9   edad                  185339 non-null  int16  
 10  Sexo                  185339 non-null  object 
 11  Ingreso por Hogar     185339 non-null  float64
 12  codigo_comuna         185339 non-null  int64  
 13  nombre_comuna         185339 non-null  object 
 14  codigo_provincia      185339 non-null  int64  
 15  

Limpiamos columnas innecesarias

In [29]:
df_final = df_final.drop(['codigo_provincia','nombre_provincia','codigo_comuna','comuna'], axis=1)


In [31]:
df_final = df_final.rename(columns={'nombre_comuna': 'Comuna'})

In [32]:
df_final.head()

,folio,id_persona,Pobreza,Nivel socioeconomico,pobreza_sinte,region,Año,Sistema Provisional,edad,Sexo,Ingreso por Hogar,Comuna
0,1.101100e+11,5,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,34,Mujer,390833.0,Iquique
1,1.101100e+11,6,No Pobreza,Medio,3,1,2020,FF.AA. y del Orden,4,Mujer,390833.0,Iquique
2,1.101100e+11,32,No Pobreza,Bajo-medio-alto,3,1,2020,Fonasa,45,Hombre,947583.0,Iquique
3,1.101100e+11,31,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),5,Mujer,947583.0,Iquique
4,1.101100e+11,30,No Pobreza,Bajo-medio-alto,3,1,2020,Ninguno (particular),19,Mujer,947583.0,Iquique


Finalmente creamos el DataFrame definitivo

In [33]:
df_final.to_csv("CASEN_PANDEMIA_POBREZA_GEOGRAFICA_2020.csv", index=False, encoding='utf-8-sig')